In [1]:
import os
import redis
import pickle
import numpy as np
import tensorflow as tf
from multiprocessing import Process

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
model = tf.keras.models.load_model('alexnet-cifar10_origin.h5')
new_output = tf.keras.layers.Softmax()(tf.keras.layers.Dense(10)(model.layers[-2].output)) # without specifying activation 
new_model = tf.keras.models.Model(inputs=model.input, outputs=new_output)
new_model.layers[-2].set_weights(model.layers[-1].get_weights())
model = new_model

# model = tf.keras.applications.ResNet50()
# new_output = tf.keras.layers.Softmax()(tf.keras.layers.Dense(1000)(model.layers[-2].output)) # without specifying activation 
# new_model = tf.keras.models.Model(inputs=model.input, outputs=new_output)
# new_model.layers[-2].set_weights(model.layers[-1].get_weights())
# model = new_model

# model = tf.keras.applications.MobileNetV2()
# new_output = tf.keras.layers.Softmax()(tf.keras.layers.Dense(1000)(model.layers[-2].output)) # without specifying activation 
# new_model = tf.keras.models.Model(inputs=model.input, outputs=new_output)
# new_model.layers[-2].set_weights(model.layers[-1].get_weights())
# model = new_model

# model = tf.keras.applications.VGG16()
# new_output = tf.keras.layers.Softmax()(tf.keras.layers.Dense(1000)(model.layers[-2].output)) # without specifying activation 
# new_model = tf.keras.models.Model(inputs=model.input, outputs=new_output)
# new_model.layers[-2].set_weights(model.layers[-1].get_weights())
# model = new_model

2022-03-27 23:44:01.840776: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-27 23:44:01.840853: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ugcpu4
2022-03-27 23:44:01.840866: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ugcpu4
2022-03-27 23:44:01.841099: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 495.29.5
2022-03-27 23:44:01.841168: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 495.29.5
2022-03-27 23:44:01.841182: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 495.29.5
2022-03-27 23:44:01.842519: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in perf

In [3]:
# import to tf2.7
def import_model(model_name, model, db_flag):
    r = redis.Redis(db=db_flag)
    w = pickle.loads(r.hget(model_name, 'weights'))
    x = pickle.loads(r.hget(model_name, 'input'))

    model.set_weights(w)
        
    return model, x

def tf27_outputs(model, x):
    # collect model intermediate layers output function
    layers_output = []
    for l in model.layers:
        layers_output.append(l.get_output_at(-1))
        
    extractor = tf.keras.Model(inputs=model.inputs, outputs=layers_output)
    o_tf = extractor.predict(x)
    
    return o_tf[1:]

def tf2torch_outputs(model, x, db_flag):
    TORCH_MODEL_DIR = 'torch_model'
    if not os.path.exists(TORCH_MODEL_DIR):
        os.mkdir(TORCH_MODEL_DIR)
        
    model_key = 0
    input_key = 0
    r = redis.Redis(db=db_flag)
    r.hset('model', model_key, pickle.dumps(model))
    r.hset('input', input_key, pickle.dumps(x))
        
    # get torch outputs for each model layer
    P = []
    for i in range(1, len(model.layers)):
        cmd = f'/data/yylaiai/anaconda3/envs/fyp_v3/bin/python get_prediction_torch.py {db_flag} {model_key} {input_key} {i}'
        p = Process(target=lambda: os.system(cmd))
        p.start()
        P.append(p)
        
    for p in P:
        p.join()
    P = []
        
    # check if all layer outputs are computed
    for i in range(1, len(model.layers)):
        if r.hget(f'predictions_{i}', 'torch') == None:
            raise Exception(f'Layer {i} not computed')

    o_torch = []
    for i in range(1, len(model.layers)):
        o_torch.append(pickle.loads(r.hget(f'predictions_{i}', 'torch')))
        
    return o_torch

In [4]:
model, x = import_model('alexnet', model, 4)

In [5]:
o_tf = tf27_outputs(model, x[None,:])

In [13]:
o_torch = tf2torch_outputs(model, x, 4)

o_torch_np = []
for o in o_torch:
    o_torch_np.append(o.detach().numpy())

INFO:tensorflow:Assets written to: ram://968f2673-3ecf-4149-ac52-7762ccd8f1d9/assets


/data/yylaiai/fyp21-22/GA/stable/get_prediction_torch.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/data/yylaiai/fyp21-22/GA/stable/get_prediction_torch.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/data/yylaiai/fyp21-22/GA/stable/get_prediction_torch.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/data/yylaiai/fyp21-22/GA/stable/get_prediction_torch.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/data/yylaiai/fyp21-22/GA/stable/get_prediction_torch.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/data/yyla

2022-03-27 23:47:40.357217: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-27 23:47:40.357314: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ugcpu4
2022-03-27 23:47:40.357329: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ugcpu4
2022-03-27 23:47:40.357642: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 495.29.5
2022-03-27 23:47:40.357712: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 495.29.5
2022-03-27 23:47:40.357727: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 495.29.5
2022-03-27 23:47:40.358428: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in perf

2022-03-27 23:47:42.261019: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-03-27 23:47:42.329371: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-03-27 23:47:42.595062: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-03-27 23:47:42.697771: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-03-27 23:47:42.779753: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-03-27 23:47:42.823400: W tensorflow/python/util/util.cc:368] Sets are not currently considered seque

2022-03-27 23:47:45.486299: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/data/yylaiai/anaconda3/envs/fyp_v3/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
/data/yylaiai/anaconda3/envs/fyp_v3/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
/data/yylaiai/anaconda3/envs/fyp_v3/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
/data/yylai

...

# Autogenerated by onnx-pytorch.

import glob
import os
import math

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision


class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self._vars = nn.ParameterDict()
    self._regularizer_params = []
    for b in glob.glob(
        os.path.join(os.path.dirname(__file__), "variables", "*.npy")):
      v = torch.from_numpy(np.load(b))
      requires_grad = v.dtype.is_floating_point or v.dtype.is_complex
      self._vars[os.path.basename(b)[:-4]] = nn.Parameter(v, requires_grad=requires_grad)
    self.n_StatefulPartitionedCall_model_conv2d_1_BiasAdd = nn.Conv2d(**{'groups': 1, 'dilation': [1, 1], 'out_channels': 96, 'padding': [0, 0], 'kernel_size': (3, 3), 'stride': [2, 2], 'in_channels': 3, 'bias': True})
    self.n_StatefulPartitionedCall_model_conv2d_1_BiasAdd.weight.data = self._vars["StatefulPartitionedCall_model_conv2d_1_Conv2D_ReadVariableOp_0"]
    self.n_

# Autogenerated by onnx-pytorch.

import glob
import os
import math

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision


class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self._vars = nn.ParameterDict()
    self._regularizer_params = []
    for b in glob.glob(
        os.path.join(os.path.dirname(__file__), "variables", "*.npy")):
      v = torch.from_numpy(np.load(b))
      requires_grad = v.dtype.is_floating_point or v.dtype.is_complex
      self._vars[os.path.basename(b)[:-4]] = nn.Parameter(v, requires_grad=requires_grad)
    self.n_StatefulPartitionedCall_model_conv2d_1_BiasAdd = nn.Conv2d(**{'groups': 1, 'dilation': [1, 1], 'out_channels': 96, 'padding': [0, 0], 'kernel_size': (3, 3), 'stride': [2, 2], 'in_channels': 3, 'bias': True})
    self.n_StatefulPartitionedCall_model_conv2d_1_BiasAdd.weight.data = self._vars["StatefulPartitionedCall_model_conv2d_1_Conv2D_ReadVariableOp_0"]
    self.n_

Traceback (most recent call last):
  File "/data/yylaiai/fyp21-22/GA/stable/get_prediction_torch.py", line 62, in <module>
    predictions = model_torch(x_torch)
  File "/data/yylaiai/anaconda3/envs/fyp_v3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "torch_model/layer_17/model.py", line 76, in forward
    StatefulPartitionedCall_model_batch_normalization_3_FusedBatchNormV3_0 = self.n_StatefulPartitionedCall_model_batch_normalization_3_FusedBatchNormV3(StatefulPartitionedCall_model_max_pooling2d_3_MaxPool_0)
  File "/data/yylaiai/anaconda3/envs/fyp_v3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/data/yylaiai/anaconda3/envs/fyp_v3/lib/python3.9/site-packages/torch/nn/modules/batchnorm.py", line 168, in forward
    return F.batch_norm(
  File "/data/yylaiai/anaconda3/envs/fyp_v3/lib/python3.9/site-packages/torch/nn/f

Traceback (most recent call last):
  File "/data/yylaiai/fyp21-22/GA/stable/get_prediction_torch.py", line 62, in <module>
    predictions = model_torch(x_torch)
  File "/data/yylaiai/anaconda3/envs/fyp_v3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "torch_model/layer_16/model.py", line 76, in forward
    StatefulPartitionedCall_model_batch_normalization_3_FusedBatchNormV3_0 = self.n_StatefulPartitionedCall_model_batch_normalization_3_FusedBatchNormV3(StatefulPartitionedCall_model_max_pooling2d_3_MaxPool_0)
  File "/data/yylaiai/anaconda3/envs/fyp_v3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/data/yylaiai/anaconda3/envs/fyp_v3/lib/python3.9/site-packages/torch/nn/modules/batchnorm.py", line 168, in forward
    return F.batch_norm(
  File "/data/yylaiai/anaconda3/envs/fyp_v3/lib/python3.9/site-packages/torch/nn/f

/data/yylaiai/fyp21-22/GA/stable/get_prediction_torch.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1_input (InputLayer)  [(None, 32, 32, 3)]      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 96)        2688      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 96)         0         
 2D)                                                             
                                                                 
 batch_normalization_1 (Batc  (None, 8, 8, 96)         384       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 256)         614656    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 256)        0     

In [14]:
from torch_model.layer_18.model import Model
m = Model()

import torchsummary
torchsummary.summary(m, (32,32,3))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 15, 15]           2,688
         MaxPool2d-2             [-1, 96, 7, 7]               0
       BatchNorm2d-3             [-1, 96, 7, 7]             192
            Conv2d-4            [-1, 256, 7, 7]         614,656
         MaxPool2d-5            [-1, 256, 3, 3]               0
       BatchNorm2d-6            [-1, 256, 3, 3]             512
            Conv2d-7            [-1, 384, 3, 3]         885,120
            Conv2d-8            [-1, 384, 3, 3]       1,327,488
            Conv2d-9            [-1, 256, 3, 3]         884,992
        MaxPool2d-10            [-1, 256, 1, 1]               0
      BatchNorm2d-11            [-1, 256, 1, 1]             512
Total params: 3,716,160
Trainable params: 3,716,160
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forw

In [15]:
for i in range(len(o_torch)):
    print(o_torch[i].shape, o_tf[i].shape)

torch.Size([1, 15, 15, 96]) (1, 16, 16, 96)
torch.Size([1, 7, 7, 96]) (1, 8, 8, 96)
torch.Size([1, 7, 7, 96]) (1, 8, 8, 96)
torch.Size([1, 7, 7, 256]) (1, 8, 8, 256)
torch.Size([1, 3, 3, 256]) (1, 3, 3, 256)
torch.Size([1, 3, 3, 256]) (1, 3, 3, 256)
torch.Size([1, 3, 3, 384]) (1, 3, 3, 384)
torch.Size([1, 3, 3, 384]) (1, 3, 3, 384)
torch.Size([1, 3, 3, 256]) (1, 3, 3, 256)
torch.Size([1, 1, 1, 256]) (1, 1, 1, 256)
torch.Size([3, 1, 1, 256]) (1, 1, 1, 256)
torch.Size([3, 256]) (1, 256)
torch.Size([3, 4096]) (1, 4096)
torch.Size([3, 4096]) (1, 4096)
torch.Size([3, 4096]) (1, 4096)
torch.Size([3, 4096]) (1, 4096)
torch.Size([3, 10]) (1, 10)
torch.Size([3, 10]) (1, 10)


In [12]:
for i in range(len(o_tf)):
    print(np.allclose(o_torch_np[i], o_tf[i]))

ValueError: operands could not be broadcast together with shapes (1,15,15,96) (1,16,16,96) 